In [1]:
import pandas as pd
import statsmodels.api as sm

from datetime import datetime
from sklearn.metrics import r2_score, explained_variance_score

In [15]:
model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['alignement_politique'] != 'autre']

Approche économétrique négative : on étudie l'écart des comportements réels à des comportements théoriques.
En l'occurrence à des normes encadrant les pratiques journalistes.
La méthode ici consiste à modéliser ces normes, puis à évaluer dans quelle mesure elles permettent de prédire les comportements réels, et à observer comment ces derniers s'en écartent. 
On s'intéresse aux R2 et aux résidus.

Dans l'idéal, les normes doivent être parfaitement connues.
Exemple de la règle des tiers, les articles se répartissent au tiers entre le gouvernement, la majorité et l'opposition.
Mais d'application délicate car elle concerne des unités fluctuant au cours du temps (les partis qui gouvernent, les partis qui soutiennent le gouvernement sans y participer, les partis qui s'opposent au gouvernement).
Et dépourvue d'utilité pour nous : ne permet pas d'étudier des outcomes spécifiques à l'extrême-droite, qui sera toujours diluée dans l'opposition.

Modèle alternatif : les journaux accordent une fraction $\alpha$ des articles à la nuance politique qui gouverne, et répartissent les articles restants entre les nuances politiques en fonction de leur poids électoral. Ce qu'on peut écrire, en notant $G_i$ la participation au gouvernement et $T_i$ les résultats à la dernière élection législative : 
$$Y_{i} = \alpha + \beta G_i + \gamma T_i$$

La difficulté est la valeur de $\alpha$ n'est pas connue *a priori* (pas d'argument dans la littérature pour étayer $\alpha = \frac{1}{3}$ par exemple). 
Il faut l'estimer à partir des données.
Risque que le raisonnement deviennent circulaire : si l'on tente de mesurer l'écart des comportements réels à une norme en définissant celle-ci à partir de ces mêmes comportements, on se retrouve en fait à commenter la performance prédictive du modèle.
Pour minimiser ce risque, il faut que le nombre de paramètres estimés soit le plus faible possible (comme dans le modèle précédent, supposant que la norme est identique pour les nuances politiques, les journaux et les périodes).

*NB : une difficulté supplémentaire pour étudier le phénomène de légitimation de l'extrême droite : des évolutions surviennent au cours du temps, soit que les normes évoluent, soit que les écarts aux normes s'amplifient. Estimer les paramètres de la norme sur toute la période conduit à sous-estimer la transformation des comportements réels. Dans ces circonstances, il vaut mieux :*
- *Estimer sur une période de référence, et appliquer la norme ainsi obtenue aux autres périodes.*
- *Estimer plusieurs normes sur plusieurs périodes, mais cela est une manière de revenir à une approche économétrique traditionnelle.*
*D'un autre côté, dans le modèle actuel, aucun paramètre n'est spécifique à l'extrême droite.*

In [39]:
######################

Premier regard sur les résultats :
- Nécessité d'ajouter une constance pour calculer correctement les R2 (permet aussi de réduire le BIC)
- Le meilleur modèle de base intègre une constante, les résultats électoraux et une indicatrice gouvernementale.
- Ce modèle fonctionne bien pour la période <2012, beaucoup moins ensuite.
- Ce modèle fonctionne pour tous les journaux, sauf *Médiapart*.
- Il semble fonctionner seulement pour les partis ayant accédé au pouvoir sur la période considérée (i.e., pas pour le centre avant 2012, pas pour la droite après 2012)
- Pour les autres partis, les performances du modèle semblent devenir moins mauvais après 2012.


*Pour les partis hors gouvernement, on pourrait simplement calculer le coefficient de corrélation entre art_share et votes_share ?*

# Estimation de la norme

In [ ]:
# Attention ci-dessous les modèles estimés n'a pas 1 mais 2 paramètres !

In [29]:
# model_data = model_data[model_data['alignement_politique'] == 'extremedroite_droiteradicale']

X = sm.add_constant(model_data[["votes_share", "government"]])
y = model_data["art_share"]
model = sm.OLS(y, X).fit(cov_type='HC3')

In [30]:
print(f"R2: {model.rsquared}\n")
print(pd.DataFrame({
    "coef": model.params,
    "p-value": model.pvalues
}))

R2: 0.7847345813750833

                 coef        p-value
const        0.064784  1.248755e-232
votes_share  1.337512   0.000000e+00
government   0.253887   0.000000e+00


In [19]:
cutoff = pd.Period('2012-06', freq='M')
X = sm.add_constant(model_data[model_data["month"] <= cutoff][["votes_share", "government"]])
y = model_data[model_data["month"] <= cutoff]["art_share"]
model = sm.OLS(y, X).fit(cov_type='HC3')

In [20]:
print(f"R2: {model.rsquared}\n")
print(pd.DataFrame({
    "coef": model.params,
    "p-value": model.pvalues
}))

R2: 0.5883388547981798

                 coef        p-value
const        0.150372  7.743671e-294
votes_share  1.059067  1.685151e-204
government   0.213364  6.166682e-206


# Etude des comportements prévus

In [31]:
model_data['pred_art_share'] = model.predict(sm.add_constant(model_data[["votes_share", "government"]]))

In [32]:
cutoff = pd.Period('2012-06', freq='M')

for period_label, period_filter in {
    'pre2012': model_data['month'] < cutoff,                                
    'post2012': model_data['month'] >= cutoff
}.items():
    subset_data = model_data[period_filter]
    print(period_label)
    print(f"{r2_score(subset_data['art_share'], subset_data['pred_art_share'])}")

pre2012
0.8675801307436324
post2012
0.5059728058563995


In [33]:
journals = model_data['journal'].unique()

for journal in journals:
    subset_data = model_data[model_data['journal'] == journal]
    print(journal)
    print(f"{r2_score(subset_data['art_share'], subset_data['pred_art_share'])}")

Le Figaro
0.8486867394846482
Le Monde
0.8342892392337119
La Croix
0.861528373965645
Libération
0.8346227699730442
Mediapart
-0.5624689213480643


In [34]:
alignments = model_data['alignement_politique'].unique()

for alignment in alignments:
    subset_data = model_data[model_data['alignement_politique'] == alignment]
    print(alignment)
    print(f"{r2_score(subset_data['art_share'], subset_data['pred_art_share'])}")

centredroite_droite
0.5973130262968649
centregauche_gauche
0.4068605143442102
extremegauche_gaucheradicale
-0.6111649389830052
centre
0.7988183208829636
extremedroite_droiteradicale
-0.3785901859737786


In [35]:
cutoff = pd.Period('2012-06', freq='M')

journals = model_data['journal'].unique()
alignments = model_data['alignement_politique'].unique()

results = {}

for period_label, period_filter in {
    'pre2012': model_data['month'] < cutoff,                                
    'post2012': model_data['month'] >= cutoff
}.items():
    results[period_label] = {}
    
    for alignment in alignments:
        results[period_label][alignment] = {}
        
        for journal in journals:
            subset_data = model_data[
                (period_filter) &
                (model_data['journal'] == journal) &
                (model_data['alignement_politique'] == alignment)]
            
            if len(subset_data) >= 3:
                results[period_label][alignment][journal] = r2_score(subset_data['art_share'], subset_data['pred_art_share'])
            else:
                results[period_label][alignment][journal] = None

In [36]:
r2_tables = {}
for period, alignements in results.items():
    r2_tables[period] = pd.DataFrame({
        alignement: {
            journal: results[period][alignement][journal]
            for journal in results[period][alignement]
        } for alignement in results[period]
    }).sort_index().T

In [37]:
r2_tables["pre2012"]

,La Croix,Le Figaro,Le Monde,Libération,Mediapart
centredroite_droite,0.803500,0.631184,0.705358,0.580185,-0.031985
centregauche_gauche,0.784026,0.614825,0.366468,0.281147,-5.524372
extremegauche_gaucheradicale,-11.822871,-17.231408,-6.225167,-5.162818,-3.258605
centre,-0.138056,-0.413072,-0.602119,-0.118666,NaN
extremedroite_droiteradicale,-3.514805,-3.663144,-2.829312,-5.043083,-4.756246


In [38]:
r2_tables["post2012"]

,La Croix,Le Figaro,Le Monde,Libération,Mediapart
centredroite_droite,-1.542454,-6.883047,-4.008233,-1.419806,-0.788268
centregauche_gauche,0.803833,0.701169,0.408641,0.521507,-1.671137
extremegauche_gaucheradicale,-0.457542,-4.424052,0.105452,0.038718,-1.642463
centre,0.661048,0.585289,0.561347,0.648283,-0.082431
extremedroite_droiteradicale,-0.577944,-1.869290,-0.461168,-0.902084,-1.457527


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# x1 and x2
x1 = model_data["votes_share"].values
x2 = model_data["government"].values
y = model_data["art_share"].values

# Objective: residual sum of squares
def objective(alpha):
    y_pred = alpha * x1 + (1 - alpha) * x2
    return np.sum((y - y_pred) ** 2)

# Minimize the objective
result = minimize(objective, x0=0.5, bounds=[(0, 1)])

# Estimated alpha
alpha_hat = result.x[0]
print(f"Estimated alpha: {alpha_hat:.4f}")

In [ ]:
# Predicted values using estimated alpha
y_pred = alpha_hat * x1 + (1 - alpha_hat) * x2

# Residual and total sum of squares
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)

# R-squared
r2 = 1 - ss_res / ss_tot
print(f"R-squared: {r2:.4f}")